In [10]:
# Libs
import json

In [11]:
# Load polygon GeoJSON file
with open(
    "../../data/opendata/masterdata/0301100100_UNITATS_ADM_POLIGONS_EPSG-4326.json",
    "r",
    encoding="utf-8",
) as f:
    bcn_masterdata_polygons = json.load(f)

# Load coordinates GeoJSON file
with open(
    "../../data/opendata/masterdata/0301100100_UNITATS_ADM_PUNTS-ESPG-4326.json",
    "r",
    encoding="utf-8",
) as f:
    bcn_masterdata_coordinates = json.load(f)

In [12]:
def load_masterdata(id: str, input_file, output_file: str):
    """
    Generate geojson files per Basic Statistical Administration (AEB),
    District, Neighborhood and Registry
    """

    # Get only `id` features (e.g.: AEB, DISTRICT..) and update
    # the previous feature id with the new id
    for feature in input_file["features"]:
        if feature["properties"]["TIPUS_UA"] == id:
            if id == "SEC_CENS":
                # Compound index: district + registry
                feature["id"] = (
                    feature["properties"]["DISTRICTE"]
                    + feature["properties"]["SEC_CENS"]
                )
            else:
                feature["id"] = int(feature["properties"][id])

    # Exclude any unrelated feature from the original geojson
    bcn_masterdata = {
        "type": "FeatureCollection",
        "features": [
            feature
            for feature in input_file["features"]
            if feature["properties"]["TIPUS_UA"] == id
        ],
    }

    # Save updated GeoJSON file
    with open(
        f"../../data/input/{output_file}.geojson", "w", encoding="utf-8"
    ) as outfile:
        json.dump(bcn_masterdata, outfile, ensure_ascii=False)


load_masterdata("AEB", bcn_masterdata_polygons, "md_aeb_polygons")
load_masterdata("BARRI", bcn_masterdata_polygons, "md_neighborhood_polygons")
load_masterdata("SEC_CENS", bcn_masterdata_polygons, "md_sec_cens_polygons")
load_masterdata("DISTRICTE", bcn_masterdata_polygons, "md_district_polygons")

load_masterdata("AEB", bcn_masterdata_coordinates, "md_aeb_coordinates")
load_masterdata("BARRI", bcn_masterdata_coordinates, "md_neighborhood_coordinates")
load_masterdata("SEC_CENS", bcn_masterdata_coordinates, "md_sec_cens_coordinates")
load_masterdata("DISTRICTE", bcn_masterdata_coordinates, "md_district_coordinates")